In [22]:
import pandas as pd
import os
import time
import csv
import re
import codecs
import requests
from bs4 import BeautifulSoup
 
    
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, \
        TimeoutException, ElementNotInteractableException, ElementClickInterceptedException

In [2]:
def text_file_to_dataframe(filename):
    # Check if the text file exists
    if os.path.exists(filename):
        
        # Read the text file and create a list of lines
        with open(filename, "r") as file:
            lines = file.readlines()
            content = [line.strip() for line in lines]

        # Create a DataFrame from the list
        df = pd.DataFrame(content, columns=['links'])
        
        return df
    else:
        print(f"{filename} does not exist.")
        return None

# Loading Links & Creating DataFrame

In [3]:
input_filename = 'final_links_again.txt'
df = text_file_to_dataframe(input_filename)

In [4]:
# new cell

df = df[:2000]

In [5]:
df.shape

(2000, 1)

# Scraper Code

In [6]:
# # S C R A P I N G   B R O W S E R
# driver = webdriver.Firefox()

In [7]:
# df['links'].iloc[4]

In [8]:
# driver.get('https://www.goodreads.com/choiceawards/best-fantasy-books-2022')

In [9]:
# button = driver.find_element_by_css_selector('button[aria-label="Book details and editions"]')

In [10]:
# button.click()

In [11]:
# close_button = driver.find_element_by_class_name('gr-iconButton')
# close_button.text

In [12]:
# close_button = driver.find_element_by_css_selector('.gr-iconButton')
# close_button.click()


In [13]:
# <span class="Button__labelItem">Book details &amp; editions</span>

In [19]:
import selenium

In [20]:
selenium.__version__


'4.8.2'

In [23]:
# Define a function to extract the publisher element text
def get_publisher_text():
    try:
        # Find the element with the EditionDetails class
        time.sleep(10)
        edition_details_element = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CLASS_NAME, 'EditionDetails')))
        
        # Find the nested element with the data-testid attribute
        publisher_element = edition_details_element.find_elements_by_css_selector(By.XPATH, '//div[data-testid="contentContainer"]')

        
        return publisher_element[1].text

    except NoSuchElementException:
        return ""
    except TimeoutException:
        print("Timeout: EditionDetails element not found within the specified time")
        return ""
    except ElementClickInterceptedException:
        return ""

In [ ]:
# S C R A P I N G   B R O W S E R
driver = webdriver.Chrome()

reviews_file_exists = os.path.isfile("reviews.csv")
print("Does Review CSV File Exists?")
print(reviews_file_exists)
print("\n")

index_file_exists = os.path.isfile("index.txt")

# checking last value of index in order to continue from last position
if index_file_exists:
    with open("index.txt", "r") as index_file:
        last_index = int(index_file.read())
else:
    last_index = 0

print("Output of last_index")
print(last_index)

# iterate over the dataframe
for index, row in df.iloc[last_index:].iterrows():
    print("Ouput of index")
    print(index)
    
    print("Ouput of row")
    print(row)

    #============================================================================================================
    # L O A D    P A G E    O N    S C R A P I N G    B R O W S E R
    #============================================================================================================
    driver.get(row['links'])
    try:
        print("Waiting for Title Element to Load....")
        title_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'BookPageTitleSection__title')))
        print("Assigning Title")
        title = title_element.text
    except TimeoutException:
        print(f"Timed out while waiting for title for book {row['links']}, moving on to next book")
        continue
    
    print("Ouput of title")
    print(title)
    
    #============================================================================================================
    # P U B L I S H E R   D A T A
    #============================================================================================================
    # First, try to find and click the button
    button_found = False
    try:
        time.sleep(10)
        button = driver.find_element(By.XPATH, '//button[aria-label="Book details and editions"]')
        

        button.click()
        button_found = True
        # Wait for the publisher element to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "EditionDetails")))
    except (NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException):
        print("Button not found or not clickable")

    # If the button is not found or not clickable, the get_publisher_text function will still run
    publisher = get_publisher_text()

    # Print the extracted publisher text
    if publisher is not None:
        print("=====================")
        print(" P U B L I S H E R ")
        print("=====================")
        print(publisher)
        print("=====================")
    else:
        print("Publisher text not found")


    #============================================================================================================
    # D E F I N I T I O N    S E C T I O N
    #============================================================================================================
    try:
        print("Waiting for Synopsis and Reviews to load....")
        time.sleep(20)
        synopsis_and_review_list = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'Formatted')))
        print(f"The variable length of book {index+1} = ")
        print(len(synopsis_and_review_list))
        synopsis_length = len(synopsis_and_review_list)
        print("Output of synopsis_length")
        print(synopsis_length)
    except TimeoutException:
        print(f"Timed out while waiting for synopsis and review list for book {title}, moving on to next book")
        continue
    
    try:
        page_count_element = driver.find_element(By.XPATH, '//p[data-testid="pagesFormat"]')
        page_count = page_count_element.text  
        
        print("Ouput of page_count")
        print(page_count)
    except NoSuchElementException:
        page_count = ""
    
    try:
        year_element = driver.find_element(By.XPATH, '//p[data-testid="publicationInfo"]')
        year = year_element.text
        
        print("Ouput of year")
        print(year)
    except NoSuchElementException:
        year = ""
    
    try:
        review_count_element = driver.find_element(By.XPATH, '//span[data-testid="reviewsCount"]')
        review_count = review_count_element.text
        
        print("Ouput of review_count")
        print(review_count)
        
        # converting string to interger with RegEx
        review_count_match = re.search(r'\d+', review_count)
        if review_count_match:
            review_count = int(review_count_match.group())
            print("Ouput of RegEx: review_count")
            print(review_count)
        else:
            review_count = 0
            print("Ouput of No_RegEx: review_count")
            print(review_count)
            
    except NoSuchElementException:
        review_count = 0
        print("Ouput of No_Count: review_count")
        print(review_count)
    
    try:
        rating_count_element = driver.find_element(By.XPATH, '//span[data-testid="ratingsCount"]')
        rating_count = rating_count_element.text 
        print("Ouput of rating_count")
        print(rating_count)
    except NoSuchElementException:
        rating_count = ""
    
    try:
        rating_element = driver.find_element(By.XPATH, '//RatingStatistics__rating')
        rating = rating_element.text
        print("Ouput of rating")
        print(rating)
        
    except NoSuchElementException:
        rating = ""
    
    try:
        genre_class_element = driver.find_element(By.XPATH, '//BookPageMetadataSection__genres')
        genres_text_element = genre_class_element.find_elements(By.XPATH, '//.BookPageMetadataSection__genreButton .Button__labelItem')
        genre = genres_text_element[0].text
        print("Ouput of genre")
        print(genre)
        
    except NoSuchElementException:
        genre = ""
    
    try:
        author_element = driver.find_element(By.XPATH, '//ContributorLink__name')('ContributorLink__name')
        author = author_element.text
        print("Ouput of author")
        print(author)

    except NoSuchElementException:
        author = ""
        
    book_reviews = [title, page_count, year, review_count, rating, rating_count, genre, author,publisher]
    
    # current row in DataFrame. create these columns
    # and add book information to current dataframe row
    df.at[index, 'page_count'] = page_count
    df.at[index, 'publisher'] = publisher
    df.at[index, 'year'] = year
    df.at[index, 'review_count'] = review_count
    df.at[index, 'rating'] = rating
    df.at[index, 'rating_count'] = rating_count
    df.at[index, 'genre'] = genre
    df.at[index, 'author'] = author

    
    # =====================
    # T E S T   P R I N T
    # =====================
    print(f"Book {index}:")
    print(title)
    print("==============")
    print(page_count)
    print("\n")
    print("==============")
    print(year)
    print("\n")
    print("==============")
    print("Total Reviews: ")
    print(review_count)
    print("\n")
    print("==============")
    print(rating)
    print("\n")
    print("==============")
    print(rating_count)
    print("\n")
    print("==============")
    print(genre)
    print("\n")
    print("==============")
    print(author)
    print("\n")
    print("==============")
    print(publisher)
    print("\n")
    # ==================================================================
    # R E V I E W S
    # ==================================================================
    # iterating 5 times for reviews offset by 3 
    # list of elements "synopsis_and_review_list" has synopsis
    # and reviews
    review_index = 3
    for i in range(5):
        if review_index < synopsis_length:
            print(f"Review {i + 1}")
            print("==============")
            print(synopsis_and_review_list[review_index].text)

            # saving current review to temporary list
            book_reviews.append(synopsis_and_review_list[review_index].text)
            review_index += 1
        else:
            book_reviews.append("")
    # ==================================================================
    # S A V I N G   P L A C E   A F T E R   E A C H   I T E R A T I O N
    # ==================================================================
    with open("reviews.csv", "a", newline='', encoding='utf-8') as reviews_file:
        csv_writer = csv.writer(reviews_file)
        if not reviews_file_exists:
            csv_writer.writerow(['Book Name', 'page_count', 'year', 'review_count', 'rating', 'rating_count', 'genre', 'author', 'publisher', 'Review 1', 'Review 2', 'Review 3', 'Review 4', 'Review 5'])
            reviews_file_exists = True
        csv_writer.writerow(book_reviews)

    # ==================================================================
    # S A V I N G   P L A C E   A F T E R   E A C H   I T E R A T I O N
    # ==================================================================
    # save place of index to file for resuming later
    with open("index.txt", "w") as index_file:
        index_file.write(str(index + 1))

# D O N E    W I T H    S C R A P E R
driver.quit()

if not reviews_file_exists:
    open("reviews.csv", "a", newline='', encoding='utf-8').close()

if not index_file_exists:
    open("index.txt", "a").close()

Does Review CSV File Exists?
False


Output of last_index
0
Ouput of index
0
Ouput of row
links    https://www.goodreads.com/book/show/257233.Rai...
Name: 0, dtype: object
Waiting for Title Element to Load....
Assigning Title
Ouput of title
Raintree County
Button not found or not clickable
Timeout: EditionDetails element not found within the specified time
 P U B L I S H E R 

Waiting for Synopsis and Reviews to load....
The variable length of book 1 = 
33
Output of synopsis_length
33
Ouput of No_Count: review_count
0
Book 0:
Raintree County






Total Reviews: 
0

















Review 1
Did they stop to think, in the midst of their gaiety and laughter, that they were passing burial places and battlegrounds of vanished peoples? Did they think that the winding river was the highway of extinct races, whose skimming light canoes did cleave the same waters in centuries long ago? Did these maidens in wide bonnets, these lads in straw skimmers and bowties, dream of aught but innocent love

Waiting for Title Element to Load....
Assigning Title
Ouput of title
The Eliots of Damerosehay #2
Pilgrim's Inn
Button not found or not clickable
Timeout: EditionDetails element not found within the specified time
 P U B L I S H E R 

Waiting for Synopsis and Reviews to load....
The variable length of book 2 = 
33
Output of synopsis_length
33
Ouput of No_Count: review_count
0
Book 1:
The Eliots of Damerosehay #2
Pilgrim's Inn






Total Reviews: 
0

















Review 1
I think I will start a bookshelf containing books that I know I will want to reread repeatedly throughout my life. The books on this shelf will be there because I've found that when I'm in a certain mood, only this book (or author sometimes) will do. It will place Gerard Manley Hopkins and Henry James next to Megan Whalen Turner and Robin McKinley with plenty of room for Dorothy Sayers. And Elizabeth Goudge will be there too. Sometimes what I like most about a book is the atmosphere it creates. What elements of a b

Waiting for Title Element to Load....
Assigning Title
Ouput of title
An A.J. Cronin Trilogy: The Northern Light, The Green Years, Shannon's Way
Button not found or not clickable
Timeout: EditionDetails element not found within the specified time
 P U B L I S H E R 

Waiting for Synopsis and Reviews to load....
The variable length of book 3 = 
3
Output of synopsis_length
3
Ouput of No_Count: review_count
0
Book 2:
An A.J. Cronin Trilogy: The Northern Light, The Green Years, Shannon's Way






Total Reviews: 
0

















Ouput of index
3
Ouput of row
links    https://www.goodreads.com/book/show/20497230-t...
Name: 3, dtype: object
Waiting for Title Element to Load....
Assigning Title
Ouput of title
The Naked and the Dead by Norman Mailer Lesson Plans
Button not found or not clickable
Timeout: EditionDetails element not found within the specified time
 P U B L I S H E R 

Waiting for Synopsis and Reviews to load....
The variable length of book 4 = 
3
Output of synopsis_length
3
Ou

Waiting for Title Element to Load....
Assigning Title
Ouput of title
Point of No Return
Button not found or not clickable
Timeout: EditionDetails element not found within the specified time
 P U B L I S H E R 

Waiting for Synopsis and Reviews to load....
The variable length of book 6 = 
33
Output of synopsis_length
33
Ouput of No_Count: review_count
0
Book 5:
Point of No Return






Total Reviews: 
0

















Review 1
I loved this book. Published in 1947, and contemporary at the time, at this point it almost qualifies as a historical novel. Charley Gray, a hard-working assistant vice-president at a small but tony New York bank, is twisting in the wind waiting for news about whether he, or a presumed rival, will be tapped for a vacant vice presidency. We spend a day in Charley's life, and then flash back to his coming of age in, and ultimately leaving, a tradition-bound small Massachusetts town in the wake of traumatic personal events. Marquand's touch in describing the struct

Assigning Title
Ouput of title
A Rage to Live
Button not found or not clickable
Timeout: EditionDetails element not found within the specified time
 P U B L I S H E R 

Waiting for Synopsis and Reviews to load....
The variable length of book 7 = 
30
Output of synopsis_length
30
Ouput of No_Count: review_count
0
Book 6:
A Rage to Live






Total Reviews: 
0

















Review 1
To my mind I am beginning to seriously consider the notion that John O'Hara is the American mid-20th century equivalent to Britain's Anthony Trollope of the 19th century. O'Hara was a prolific writer of short-stories, novellas, and novels from the late-1920s up until his death in 1970. He was incredibly adept at portraying all walks of life in the Pennsylvania coal country that he grew up in, or the streets and nightclubs of New York City, as well as on the film lots and studios of Hollywood. His real skill though, is his ability to describe and characterize the relationships between men and women.

O'Hara'

...

Waiting for Title Element to Load....
Assigning Title
Ouput of title
The Group
Button not found or not clickable
Timeout: EditionDetails element not found within the specified time
 P U B L I S H E R 

Waiting for Synopsis and Reviews to load....
The variable length of book 46 = 
33
Output of synopsis_length
33
Ouput of No_Count: review_count
0
Book 45:
The Group






Total Reviews: 
0

















Review 1
I can remember my Dad's married sisters discussing this book (they were voracious readers always) in the 1960's. I was determined to read it and finally got hold of it in 1967 when I was studying to be a Catholic priest. My Student Director immediately confiscated it, so I knew its reputation was still going strong.(He didn't see my two volumes of Nietzsche I'd also bought with money my Mum had given me for my 20th birthday - I'd only bought them because I'd already seen him confiscate a Nietzsche on the grounds that it could destroy one's faith and I was already seeing large h

Waiting for Title Element to Load....
Assigning Title
Ouput of title
The Rector of Justin
Button not found or not clickable
Timeout: EditionDetails element not found within the specified time
 P U B L I S H E R 

Waiting for Synopsis and Reviews to load....
The variable length of book 47 = 
33
Output of synopsis_length
33
Ouput of No_Count: review_count
0
Book 46:
The Rector of Justin






Total Reviews: 
0

















Review 1
The author of The Rector of Justin, Louis Auchincloss, is the last novelist of “Old Money” in America, and may be considered as the inheritor of the tradition of Henry James and Edith Wharton. He possesses neither the genius of James nor the subtlety of Wharton, but he writes with clarity, builds an exemplary novel, and—being a product of Old Money himself (of the New York variety)—explores a world he knows well with an affection tempered by understanding.

presents us with the life of the founder of St. Justin Martyr, a prestigious East Coast Episcopal sc

Waiting for Title Element to Load....
Assigning Title
Ouput of title
The Spy Who Came In From the Cold by John le Carré: Summary Study Guide
Button not found or not clickable
Timeout: EditionDetails element not found within the specified time
 P U B L I S H E R 

Waiting for Synopsis and Reviews to load....
The variable length of book 48 = 
20
Output of synopsis_length
20
Ouput of No_Count: review_count
0
Book 47:
The Spy Who Came In From the Cold by John le Carré: Summary Study Guide






Total Reviews: 
0

















Review 1
I had always wanted to read the book after having seen the film. It was a very long time afterward that I had finally read it. John Le Carre is a superb writer, this particular book being one of suspense.
Review 2
John le Carré
Review 3
This is probably the first best and possibly most accurate cold war spy book, the one that launched a genre. Few authors get to invent a new category (mockumentary, etc.) but Le Carre does so here.

Lays out the foundation 

Waiting for Title Element to Load....
Assigning Title
Ouput of title
The Source
Button not found or not clickable
Timeout: EditionDetails element not found within the specified time
 P U B L I S H E R 

Waiting for Synopsis and Reviews to load....
The variable length of book 51 = 
33
Output of synopsis_length
33
Ouput of No_Count: review_count
0
Book 50:
The Source






Total Reviews: 
0

















Review 1
I can remember reading this thing for hours each night curled up in bed when I was 13 years old. It basically began my career in archaeology (fingers crossed for grad school apps). Based on Tel Megiddo, it tells the story of a fictional Tel from its birth to its excavation 12,000 years later. It follows the path of a single family against many gods as it tells the story of Canaanite, Hebrew, Greek, Roman, Ottoman, and Israeli civilization. While some of the material is dated, it nevertheless remains one of the most creative and well-written books I've ever had the pleasure of

Waiting for Title Element to Load....
Assigning Title
Ouput of title
Valley of the Dolls by Jacqueline Susann Lesson Plans
Button not found or not clickable
